# Build and Register an R-Kernel Image

Please ensure you have followed the instructions on the [prerequisites](../PRE-REQS.md) readme.

Those pre-requisites provide the IAM Policy permissions to run the commands below.

## Install the Sagemaker Image Build CLI

Install the [sagemaker-studio-image-build](https://pypi.org/project/sagemaker-studio-image-build/) python library to enable us to run the `sm-docker` build CLI.

In [ ]:
!pip install sagemaker-studio-image-build

## Build the image using sm-docker

This command will build the docker image inside CodeBuild, then upload it to ECR inside a new image repository. There are other parameters that will permit you to use existing repositories.

If you want to understand or modify the contents of the Sagemaker Studio Kernel itself, you will need to inspect the contents of the [Dockerfile](Dockerfile).

The build process will take approximately 9 minutes to complete, logs will emmited to the cell th show progress.

In [ ]:
!sm-docker build .

## Register the Image with Sagemaker as a Kernel

When the process above finishes executing the final line should give you a link to your image inside ECR.

Copy the value after ```Image URI:```

You will also need your SageMaker Studio `ROLE_ARN`.  You can copy this role from the SageMaker Studio Summary as depicted below.

![SageMaker Studio Summary](sagemaker-studio-execution-role.png)

First we create a Named Sagemaker Kernel Image which we can use to register our custom image.

In [ ]:
!aws sagemaker create-image \
    --image-name my-custom-r-kernel \
    --role-arn "<ROLE_ARN>"

We then add our newly created image as the first version of the Sagemaker Image

In [ ]:
!aws sagemaker create-image-version \
    --image-name my-custom-r-kernel \
    --base-image "<IMAGE_URI>"

Now we create a configuration for kernel images.

If you need to change some settings then modify the file [pp-image-config-input.json](pp-image-config-input.json)


In [ ]:
!aws sagemaker create-app-image-config \
    --cli-input-json file://app-image-config-input.json

## Register the Kernel with your Studio Domain

Copy your `DOMAIN_ID` from the output below.

In [ ]:
!aws sagemaker list-domains

If you have changed the name of your custom kernel then edit the file [default-user-settings.json](default-user-settings.json)

In [ ]:
!aws sagemaker update-domain --domain-id "<DOMAIN_ID>" \
    --cli-input-json file://default-user-settings.json

You have succesfully built and registered your custom image.  You can now launch the [r_xgboost_batch_transform.ipynb](r_xgboost_batch_transform.ipynb) notebook selecting the `my-custom-r-kernel`.

You can also Use the keyboard shortcut `Ctrl + Shift + L` to open Studio Launcher to create a new notebook with this R kernel:

![Launcher Select Image](launcher-select-image.png)